# Analyze chest X-ray dataset

In [1]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
savepath = "dataset/precomputed/chest_xray"

## Load precomputed graph and lens

In [2]:
labels = np.load(f"{savepath}/labels.npy")
with open(f"{savepath}/train_nodes.txt","r") as f:
    lines = f.readlines()
    train_nodes = [int(i.strip()) for i in lines]
with open(f"{savepath}/test_nodes.txt","r") as f:
    lines = f.readlines()
    test_nodes = [int(i.strip()) for i in lines]
with open(f"{savepath}/test_img_names.txt","r") as f:
    lines = f.readlines()
    test_img_names = [i.strip() for i in lines]
ei,ej,e = [],[],[]
with open(f"{savepath}/edge_list.txt","r") as f:
    lines = f.readlines()
    num_nodes = int(lines[0].strip().split(' ')[0])
    for line in lines[1::]:
        line = line.strip().split(' ')
        ei.append(int(line[0]))
        ej.append(int(line[1]))
        e.append(int(line[2]))
G = sp.csr_matrix((e,(ei,ej)),(num_nodes,num_nodes))
preds = np.load(f"{savepath}/prediction_lens.npy")
extra_lens = np.load(f"{savepath}/extra_lens.npy")
pred_labels = np.argmax(preds,1)

## Compute Reeb graph and estimated errors

In [3]:
from GTDA.GTDA_utils import compute_reeb, NN_model
from GTDA.GTDA import GTDA

nn_model = NN_model()
nn_model.preds = preds
nn_model.labels = labels
nn_model.A = G
nn_model.train_mask = np.zeros(G.shape[0])
nn_model.train_mask[train_nodes] = 1
nn_model.val_mask = np.zeros(G.shape[0])
nn_model.test_mask = np.zeros(G.shape[0])
nn_model.test_mask[test_nodes] = 1
smallest_component = 50
overlap = 0.005
labels_to_eval = list(range(preds.shape[1]))
GTDA_record = compute_reeb(GTDA,nn_model,labels_to_eval,smallest_component,overlap,extra_lens=extra_lens,
    node_size_thd=5,reeb_component_thd=5,nprocs=10,device='cuda',nsteps_preprocess=10)

Preprocess lens..
Merge reeb nodes...
Build reeb graph...
Total time for building reeb graph is 29.21596622467041 seconds
Compute mixing rate for each sample


## Compare with expert labels

In [6]:
from GTDA.GTDA_utils import find_components
import pandas as pd
from collections import Counter

n = nn_model.preds.shape[0]
pred_labels = np.argmax(nn_model.preds,1)
gtda = GTDA_record['gtda']
g_reeb = GTDA_record['g_reeb']
reeb_components = find_components(g_reeb,size_thd=0)[1]
reeb_components_to_nodes = {}
for i,reeb_component in enumerate(reeb_components):
    nodes = []
    for reeb_node in reeb_component:
        nodes += gtda.final_components_filtered[gtda.filtered_nodes[reeb_node]]
    if len(nodes) > 0:
        reeb_components_to_nodes[i] = np.unique(nodes)

node_to_component_id = np.array([-1]*n)
for i,component in reeb_components_to_nodes.items():
    for node in component:
        node_to_component_id[node] = i

# 'test_labels.csv' can be obtained from 'https://cloud.google.com/healthcare-api/docs/resources/public-datasets/nih-chest'.
pathDirData = 'dataset/chest_xray'
expert_labels = pd.read_table(f'{pathDirData}/all_findings_expert_labels/test_labels.csv',delimiter=',')
expert_tested_imgs = expert_labels['Image ID'].values
expert_labels = expert_labels['Abnormal'].values
expert_labels[np.nonzero(expert_labels=='NO')] = 0
expert_labels[np.nonzero(expert_labels=='YES')] = 1
expert_labels = expert_labels.astype(np.int64)
test_img_ids = {}
for i,img in enumerate(test_img_names):
    test_img_ids[img] = test_nodes[i]

expert_tested_imgs = np.array([test_img_ids[i] for i in expert_tested_imgs])
expert_tested_incorrect_imgs = expert_tested_imgs[np.nonzero(nn_model.labels[expert_tested_imgs] != expert_labels)[0]]
print("Type\tExpert_Labels_in_Component\tIncorrect_by_Experts\tFlagged_as_Problematic\tPrecision\tRecall")
#%%
thd = 0.5
all_estimate = []
all_labels = []
cnt = Counter([node_to_component_id[k] for k in expert_tested_incorrect_imgs])
cnt_experts = Counter([node_to_component_id[k] for k in expert_tested_imgs])
# Single component
component_indices = []
all_tp = 0
all_num_total = 0
all_num_pos = 0
all_num_experts = 0
for i,k in cnt.items(): 
    if k >= 3:
        component_indices.append(i)
for component_index,num_pos in cnt.most_common(len(component_indices)):
    tp = 0
    num_total = 0
    num_experts = 0
    for i in reeb_components_to_nodes[component_index]:
        if i in set(expert_tested_imgs):
            num_experts += 1
            error = int(nn_model.labels[i]!=pred_labels[i])
            num_total += (np.abs(error-gtda.sample_colors_mixing[i])>thd)
            tp += ((i in set(expert_tested_incorrect_imgs)) * (np.abs(error-gtda.sample_colors_mixing[i])>thd))
            all_estimate.append(np.abs(error-gtda.sample_colors_mixing[i]))
            all_labels.append(int(i in set(expert_tested_incorrect_imgs)))
    all_tp += tp
    all_num_total += num_total
    all_num_pos += num_pos
    all_num_experts += num_experts
    print(f"Single_Component\t{num_experts}\t{num_pos}\t{num_total}\t{round(tp/num_total,2)}\t{round(tp/num_pos,2)}")
# Components with 2 incorrect expert labels
component_indices = []
for i,k in cnt.items(): 
    if k == 2:
        component_indices.append(i)
tp = 0
num_total = 0
num_pos = 0
num_experts = 0
for component_index in component_indices:
    num_pos += cnt[component_index]
    for i in reeb_components_to_nodes[component_index]:
        if i in set(expert_tested_imgs):
            num_experts += 1
            error = int(nn_model.labels[i]!=pred_labels[i])
            num_total += (np.abs(error-gtda.sample_colors_mixing[i])>thd)
            tp += ((i in set(expert_tested_incorrect_imgs)) * (np.abs(error-gtda.sample_colors_mixing[i])>thd))
            all_estimate.append(np.abs(error-gtda.sample_colors_mixing[i]))
            all_labels.append(int(i in set(expert_tested_incorrect_imgs)))
all_tp += tp
all_num_total += num_total
all_num_pos += num_pos
all_num_experts += num_experts
print(f"Components_with_2_incorrect_labels\t{num_experts}\t{num_pos}\t{num_total}\t{round(tp/num_total,2)}\t{round(tp/num_pos,2)}")
# Components with 1 incorrect expert label
component_indices = []
for i,k in cnt.items(): 
    if k == 1:
        component_indices.append(i)
tp = 0
num_total = 0
num_pos = 0
num_experts = 0
for component_index in component_indices:
    num_pos += cnt[component_index]
    for i in reeb_components_to_nodes[component_index]:
        if i in set(expert_tested_imgs):
            num_experts += 1
            error = int(nn_model.labels[i]!=pred_labels[i])
            num_total += (np.abs(error-gtda.sample_colors_mixing[i])>thd)
            tp += ((i in set(expert_tested_incorrect_imgs)) * (np.abs(error-gtda.sample_colors_mixing[i])>thd))
            all_estimate.append(np.abs(error-gtda.sample_colors_mixing[i]))
            all_labels.append(int(i in set(expert_tested_incorrect_imgs)))
all_tp += tp
all_num_total += num_total
all_num_pos += num_pos
all_num_experts += num_experts
print(f"Components_with_1_incorrect_label\t{num_experts}\t{num_pos}\t{num_total}\t{round(tp/num_total,2)}\t{round(tp/num_pos,2)}")
# Components with 0 incorrect expert label
component_indices = []
for i,k in cnt_experts.items(): 
    if i not in cnt:
        component_indices.append(i)
tp = 0
num_total = 0
num_pos = 0
num_experts = 0
for component_index in component_indices:
    num_pos += cnt[component_index]
    for i in reeb_components_to_nodes[component_index]:
        if i in set(expert_tested_imgs):
            num_experts += 1
            error = int(nn_model.labels[i]!=pred_labels[i])
            num_total += (np.abs(error-gtda.sample_colors_mixing[i])>thd)
            tp += ((i in set(expert_tested_incorrect_imgs)) * (np.abs(error-gtda.sample_colors_mixing[i])>thd))
            all_estimate.append(np.abs(error-gtda.sample_colors_mixing[i]))
            all_labels.append(int(i in set(expert_tested_incorrect_imgs)))
all_tp += tp
all_num_total += num_total
all_num_pos += num_pos
all_num_experts += num_experts
print(f"Components_with_0_incorrect_label\t{num_experts}\t{num_pos}\t{num_total}\t{round(tp/num_total,2)}\tNaN")

Type	Expert_Labels_in_Component	Incorrect_by_Experts	Flagged_as_Problematic	Precision	Recall
Single_Component	53	18	17	0.82	0.78
Single_Component	10	5	5	1.0	1.0
Single_Component	9	5	4	0.25	0.2
Single_Component	19	4	7	0.57	1.0
Single_Component	9	4	5	0.8	1.0
Single_Component	10	4	3	0.33	0.25
Single_Component	7	4	2	1.0	0.5
Single_Component	8	4	5	0.6	0.75
Single_Component	14	4	4	1.0	1.0
Single_Component	4	4	2	1.0	0.5
Single_Component	7	4	3	0.33	0.25
Single_Component	10	3	2	0.0	0.0
Single_Component	6	3	1	0.0	0.0
Single_Component	4	3	2	0.5	0.33
Single_Component	6	3	3	0.33	0.33
Single_Component	3	3	2	1.0	0.67
Single_Component	5	3	3	1.0	1.0
Single_Component	5	3	2	0.5	0.33
Single_Component	8	3	5	0.4	0.67
Single_Component	7	3	4	0.5	0.67
Single_Component	19	3	8	0.25	0.67
Single_Component	9	3	8	0.38	1.0
Single_Component	8	3	3	0.33	0.33
Single_Component	8	3	4	0.5	0.67
Components_with_2_incorrect_labels	135	56	50	0.74	0.66
Components_with_1_incorrect_label	219	67	78	0.5	0.58
Components_with_0_incorr

In [7]:
from sklearn.metrics import roc_auc_score

print("AUC score is:",roc_auc_score(all_labels,all_estimate))

AUC score is: 0.752949377949378
